In [190]:
from string import punctuation
import re

In [191]:
# import data

feng = open("../Data/Input/LangId.train.English", 'r', encoding='utf8')
ffra = open("../Data/Input/LangId.train.French", 'r', encoding='utf8')
fita = open("../Data/Input/LangId.train.Italian", 'r', encoding='utf8')

eng_string = feng.read(); eng_string = eng_string.lower();
fra_string = ffra.read(); fra_string = fra_string.lower();
ita_string = fita.read(); ita_string = ita_string.lower();

feng.close();
ffra.close();
fita.close();

In [192]:
def clean_text(data):
    for i in range(len(data)):
        if data[i] == '\'':
            continue
    data = data.lower()

    # punctuation and numbers
    for char in punctuation:
        if char == '\'': 
            continue
        data = data.replace(char, ' ')
    for num in '1234567890':
        data = data.replace(num, '')

    # whitespace
    data = data.strip()
    data = data.replace("\n", " ")
    data = data.replace("   ", " ")
    data = data.replace("  ", " ")
    # data = " ".join(data.split())
    # data = data.replace(" \'", "\'")
    # data = data.replace("\' ", "\'")
    return data

                   
'''
if i is a space and i+1 is a comma/apostrophe/semicolon/, continue and do nothing else
if i+1 is . and i+2 is \n, change . to </s> and \n to <s> as long as there's a letter after it
    if i is </s> and i+1 is \n, change \n to <s> as long as there's a letter on the next line
if i find "' - s", add '\'',s to the dict and move i forward far enough
THEY USED A DASH TO MARK POSSESSIVES

i might not actually care about punctuation other than apostrophes
    i want english to have a fuckton of 's and s'
    i want french to have a fuckton of j' and t'
    i want spanish to literally have 0 of them
    italian should have l' but so should french
'''
print("")

In [193]:
# make bigrams
def make_bigram(data):
    bigram_count = {} # make a dictionary that maps bigrams to their counts

    data = data.split(" ")

    # MAKING nGRAMS
    for i in range(len(data) - 1) :
        if (data[i], data[i+1]) in bigram_count:            # if it's already counted, increment
            bigram_count[(data[i], data[i + 1])] += 1
        else:
            bigram_count[(data[i], data[i + 1])] = 1        # if it's new, put it in as 1
        
    return bigram_count

#make unigrams
def make_unigram(data):
    unigram_count = {} # make a dictionary that maps bigrams to their counts

    data = data.split(" ")

    # MAKING nGRAMS
    for i in data:
        if i in unigram_count:            # if it's already counted, increment
            unigram_count[i] += 1
        else:
            unigram_count[i] = 1          # if it's new, put it in as 1

    return unigram_count


In [194]:
# turing smooth
def turing_smooth(unigrams, bigrams) :
    max_possible = len(unigrams)**2
    zero_counts  = max_possible - len(bigrams) # the opposite of set overlaps, i forgot the word for that
    
    b_freq_dict = {}                        # make frequency dictionary of bigrams
    for val in bigrams.values() :
        if val in b_freq_dict : b_freq_dict[val] += 1
        else :                  b_freq_dict[val] = 1
                                            
    u_freq_dict = {}                        # make unigram freq dict
    for val in unigrams.values() :
        if val in u_freq_dict : u_freq_dict[val] += 1
        else :                  u_freq_dict[val] = 1
        
    b_freq_dict[0] = zero_counts            # set 0s
    u_freq_dict[0] = 1


    # go thru to find the first frequency that doesn't have anything in it
    bf_keys = list(b_freq_dict.keys())
    bf_keys.sort()
    uf_keys = list(u_freq_dict.keys())
    uf_keys.sort()

    b_threshold = None
    u_threshold = None

    for i,u in enumerate(uf_keys):
        # print("enum ", i,u)
        if i != u:
            u_threshold = uf_keys[i-1]
            break
    for i,b in enumerate(bf_keys):
        if i != b:
            b_threshold = bf_keys[i-1]
            break

    return u_freq_dict,u_threshold,b_freq_dict,b_threshold

print("")

In [195]:
from math import log

# takes in the results from turing smooth and gives a line at a time a score.
def score_line(line, unigram, uf_dict, u_threshold, bigram, bf_dict, b_threshold):
    score = 0
    # n = 1
    # V = len(unigram)
    line = line.split(" ")
    for i in range(len(line) - 1) :
        b_count = 0
        u_count = 0
        if (line[i], line[i+1]) in bigram:
            b_count = bigram[(line[i], line[i+1])]
            # print(b_count)
        if line[i] in unigram:
            u_count = unigram[line[i]]
            # print(u_count)

        # print(type(b_count, type(sum(bigram.values()))))
        numer = b_count / int(sum(bigram.values()))
        if b_count < b_threshold:
            numer = ((b_count + 1) * bf_dict[b_count + 1] / (bf_dict[b_count] * sum(bigram.values())))
        
        denom = u_count / int(sum(bigram.values()))
        if u_count < u_threshold:
            denom = ((u_count + 1) * uf_dict[u_count + 1] / (uf_dict[u_count] * sum(unigram.values())))


        score += log(numer/denom)
    return score   



In [198]:
# open the file, make/clean all the strings, turing smooth them all, and find the highest score for each line.

test_file = open("../Data/Validation/LangId.test", "r")

test = test_file.read()

test = test.split("\n")

test_file.close();

eng_bigram = make_bigram(clean_text(eng_string))
fra_bigram = make_bigram(clean_text(fra_string))
ita_bigram = make_bigram(clean_text(ita_string))
eng_unigram = make_unigram(clean_text(eng_string))
fra_unigram = make_unigram(clean_text(fra_string))
ita_unigram = make_unigram(clean_text(ita_string))

eng_uf_dict,eng_u_threshold,eng_bf_dict,eng_b_threshold = turing_smooth(eng_unigram, eng_bigram)
fra_uf_dict,fra_u_threshold,fra_bf_dict,fra_b_threshold = turing_smooth(fra_unigram, fra_bigram)
ita_uf_dict,ita_u_threshold,ita_bf_dict,ita_b_threshold = turing_smooth(ita_unigram, ita_bigram)

output = open("../Data/Output/wordLangId2.out", "w")

i = 1
for line in test :
    clean_line = clean_text(line)
    eng_score = score_line(clean_line,eng_unigram,eng_uf_dict,eng_u_threshold,eng_bigram,eng_bf_dict,eng_b_threshold)
    fra_score = score_line(clean_line,fra_unigram,fra_uf_dict,fra_u_threshold,fra_bigram,fra_bf_dict,fra_b_threshold)
    ita_score = score_line(clean_line,ita_unigram,ita_uf_dict,ita_u_threshold,ita_bigram,ita_bf_dict,ita_b_threshold)
    # print("english:", eng_score, "french:", fra_score, "italian:", ita_score)

    if eng_score >= fra_score and eng_score >= ita_score :
        output.write(str(i) + " English")
    elif fra_score >= eng_score and fra_score >= ita_score :
        output.write(str(i) + " French")
    else :
        output.write(str(i) + " Italian")
    if (i < 300): output.write("\n")
    i += 1

output.close()


In [197]:
# comparison function


def comp(out_file, sol_file) :
    out = open(out_file, "r")
    sol = open(sol_file, "r")
    num = 0
    out = out.readlines()
    sol = sol.readlines()
    for i in range(len(out)) :
        if (out[i] == sol[i]): num += 1
        else: print(out[i], sol[i])
        
    return f'{num}/{len(out)} : {round((num/len(out))*100, 2)}%'

print(comp("../Data/Output/wordLangId2.out", "../Data/Validation/labels.sol"))

300/300 : 100.0%
